## Product Sales Prediction

Given *data about products being sold*, let's try to predict if a given product will have been **sold in the last six months**.

We will use a random forest classification model to make our predictions.

Data source: https://www.kaggle.com/datasets/flenderson/sales-analysis

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, f1_score

In [2]:
data = pd.read_csv('archive/SalesKaggle3.csv')
data

,Order,File_Type,SKU_number,SoldFlag,SoldCount,MarketingType,ReleaseNumber,New_Release_Flag,StrengthFactor,PriceReg,ReleaseYear,ItemCount,LowUserPrice,LowNetPrice
0,2,Historical,1737127,0.0,0.0,D,15,1,6.827430e+05,44.99,2015,8,28.97,31.84
1,3,Historical,3255963,0.0,0.0,D,7,1,1.016014e+06,24.81,2005,39,0.00,15.54
2,4,Historical,612701,0.0,0.0,D,0,0,3.404640e+05,46.00,2013,34,30.19,27.97
3,6,Historical,115883,1.0,1.0,D,4,1,3.340110e+05,100.00,2006,20,133.93,83.15
4,7,Historical,863939,1.0,1.0,D,2,1,1.287938e+06,121.95,2010,28,4.00,23.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198912,208023,Active,109683,NaN,NaN,D,7,1,2.101869e+05,72.87,2006,54,8.46,60.59
198913,208024,Active,416462,NaN,NaN,D,8,1,4.555041e+05,247.00,2009,65,8.40,74.85
198914,208025,Active,658242,NaN,NaN,S,2,1,1.692746e+05,50.00,2012,23,23.98,32.62
198915,208026,Active,2538340,NaN,NaN,S,2,1,3.775266e+05,46.95,2001,23,27.42,37.89


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198917 entries, 0 to 198916
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Order             198917 non-null  int64  
 1   File_Type         198917 non-null  object 
 2   SKU_number        198917 non-null  int64  
 3   SoldFlag          75996 non-null   float64
 4   SoldCount         75996 non-null   float64
 5   MarketingType     198917 non-null  object 
 6   ReleaseNumber     198917 non-null  int64  
 7   New_Release_Flag  198917 non-null  int64  
 8   StrengthFactor    198917 non-null  float64
 9   PriceReg          198917 non-null  float64
 10  ReleaseYear       198917 non-null  int64  
 11  ItemCount         198917 non-null  int64  
 12  LowUserPrice      198917 non-null  float64
 13  LowNetPrice       198917 non-null  float64
dtypes: float64(6), int64(6), object(2)
memory usage: 21.2+ MB


### Preprocessing

In [4]:
df = data.copy()

In [5]:
df

,Order,File_Type,SKU_number,SoldFlag,SoldCount,MarketingType,ReleaseNumber,New_Release_Flag,StrengthFactor,PriceReg,ReleaseYear,ItemCount,LowUserPrice,LowNetPrice
0,2,Historical,1737127,0.0,0.0,D,15,1,6.827430e+05,44.99,2015,8,28.97,31.84
1,3,Historical,3255963,0.0,0.0,D,7,1,1.016014e+06,24.81,2005,39,0.00,15.54
2,4,Historical,612701,0.0,0.0,D,0,0,3.404640e+05,46.00,2013,34,30.19,27.97
3,6,Historical,115883,1.0,1.0,D,4,1,3.340110e+05,100.00,2006,20,133.93,83.15
4,7,Historical,863939,1.0,1.0,D,2,1,1.287938e+06,121.95,2010,28,4.00,23.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198912,208023,Active,109683,NaN,NaN,D,7,1,2.101869e+05,72.87,2006,54,8.46,60.59
198913,208024,Active,416462,NaN,NaN,D,8,1,4.555041e+05,247.00,2009,65,8.40,74.85
198914,208025,Active,658242,NaN,NaN,S,2,1,1.692746e+05,50.00,2012,23,23.98,32.62
198915,208026,Active,2538340,NaN,NaN,S,2,1,3.775266e+05,46.95,2001,23,27.42,37.89


In [6]:
# Only use historical data
df = df.query('File_Type == "Historical"')
df

,Order,File_Type,SKU_number,SoldFlag,SoldCount,MarketingType,ReleaseNumber,New_Release_Flag,StrengthFactor,PriceReg,ReleaseYear,ItemCount,LowUserPrice,LowNetPrice
0,2,Historical,1737127,0.0,0.0,D,15,1,682743.0,44.99,2015,8,28.97,31.84
1,3,Historical,3255963,0.0,0.0,D,7,1,1016014.0,24.81,2005,39,0.00,15.54
2,4,Historical,612701,0.0,0.0,D,0,0,340464.0,46.00,2013,34,30.19,27.97
3,6,Historical,115883,1.0,1.0,D,4,1,334011.0,100.00,2006,20,133.93,83.15
4,7,Historical,863939,1.0,1.0,D,2,1,1287938.0,121.95,2010,28,4.00,23.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75991,85102,Historical,863599,0.0,0.0,S,2,1,119056.0,64.99,2008,34,32.49,53.00
75992,85103,Historical,614304,0.0,0.0,S,6,1,1115312.0,21.50,2004,53,47.99,21.97
75993,85104,Historical,109021,0.0,0.0,S,7,1,1315446.0,142.75,2006,19,47.99,53.99
75994,85105,Historical,658242,0.0,0.0,S,2,1,877788.0,50.00,2012,21,27.07,32.62


In [7]:
{column: df[column].unique() for column in df.select_dtypes('object').columns}

{'File_Type': array(['Historical'], dtype=object),
 'MarketingType': array(['D', 'S'], dtype=object)}

In [8]:
# Drop unused columns
df = df.drop(['Order', 'SoldCount', 'File_Type', 'SKU_number'], axis=1)
df

,SoldFlag,MarketingType,ReleaseNumber,New_Release_Flag,StrengthFactor,PriceReg,ReleaseYear,ItemCount,LowUserPrice,LowNetPrice
0,0.0,D,15,1,682743.0,44.99,2015,8,28.97,31.84
1,0.0,D,7,1,1016014.0,24.81,2005,39,0.00,15.54
2,0.0,D,0,0,340464.0,46.00,2013,34,30.19,27.97
3,1.0,D,4,1,334011.0,100.00,2006,20,133.93,83.15
4,1.0,D,2,1,1287938.0,121.95,2010,28,4.00,23.99
...,...,...,...,...,...,...,...,...,...,...
75991,0.0,S,2,1,119056.0,64.99,2008,34,32.49,53.00
75992,0.0,S,6,1,1115312.0,21.50,2004,53,47.99,21.97
75993,0.0,S,7,1,1315446.0,142.75,2006,19,47.99,53.99
75994,0.0,S,2,1,877788.0,50.00,2012,21,27.07,32.62


In [9]:
df.isna().sum()

SoldFlag            0
MarketingType       0
ReleaseNumber       0
New_Release_Flag    0
StrengthFactor      0
PriceReg            0
ReleaseYear         0
ItemCount           0
LowUserPrice        0
LowNetPrice         0
dtype: int64

In [10]:
# Shuffle data
df = df.sample(frac=1.0, random_state=1)

In [11]:
# Split the data into X and y
y = df['SoldFlag']
X = df.drop('SoldFlag', axis=1)

In [12]:
X

,MarketingType,ReleaseNumber,New_Release_Flag,StrengthFactor,PriceReg,ReleaseYear,ItemCount,LowUserPrice,LowNetPrice
37862,S,12,1,545082.0,96.67,2011,12,73.74,101.33
35304,S,2,1,4273940.0,58.00,2002,32,85.60,23.98
26138,D,9,1,165834.0,76.95,2011,48,75.57,42.67
52327,S,22,1,79220.0,54.25,2012,31,36.47,22.49
6038,D,8,1,80014.0,38.99,2008,62,153.24,69.43
...,...,...,...,...,...,...,...,...,...
20609,D,8,1,40841.0,103.24,2010,48,99.50,115.55
21440,D,0,0,1611172.0,86.64,2011,19,55.19,78.38
73349,S,2,1,1628317.0,69.99,2004,43,4.02,30.43
50057,S,2,1,1660915.0,44.00,2004,32,34.51,10.12


In [13]:
y.value_counts(), y.mean()

(SoldFlag
 0.0    63000
 1.0    12996
 Name: count, dtype: int64,
 np.float64(0.17100900047370915))

### Building Pipeline

In [14]:
def build_pipeline():
    binary_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(sparse_output=False, drop='if_binary'))
    ])
        
    nominal_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
    ])
    
    preprocessor = ColumnTransformer(transformers=[
        ('binary', binary_transformer, ['MarketingType']),
        ('nominal', nominal_transformer, ['ReleaseNumber'])
    ], remainder='passthrough')

    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', RandomForestClassifier(random_state=1))
    ])

    return model

### Training/Validation

In [15]:
accs = []
f1s = []

kf = KFold(n_splits=5)

for train_idx, test_idx in kf.split(X):
    X_train = X.iloc[train_idx, :]
    X_test = X.iloc[test_idx, :]
    y_train = y.iloc[train_idx]
    y_test = y.iloc[test_idx]

    # Address class imbalance (undersampling majority class and oversampling minority class)
    # Under sampling
    num_samples = int(y_train.value_counts().mean())
    majority_indices = y_train[y==0.0].index
    samples_to_drop = y_train[majority_indices].sample(num_samples, random_state=1).index
    X_train = X_train.drop(samples_to_drop, axis=0)
    y_train = y_train.drop(samples_to_drop, axis=0)
    # Over sampling
    oversampler = RandomOverSampler(random_state=1)
    X_train, y_train = oversampler.fit_resample(X_train, y_train)
    
    model = build_pipeline()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, pos_label=1.0)

    accs.append(acc)
    f1s.append(f1)

acc = np.mean(accs)
f1 = np.mean(f1s)

print("Accuracy: {:.2f}%".format(acc * 100))
print("F1-Score: {:.5f}".format(f1))

Accuracy: 76.86%
F1-Score: 0.43570


In [16]:
from sklearn.metrics import confusion_matrix

In [17]:
confusion_matrix(y_test, y_pred)

array([[10345,  2241],
       [ 1236,  1377]])